In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
[V1]
* Resolution: Resized to 512x512 from 768x768
* Extract cell masks and create individual cell images (512x512)
* No random crop
* No TTA
* Update normalization mean and std with 2021 training and test sets

[V2]
* Use INTER_AREA for resize

[V3]
* Refactor to use less memory


Note: HPA-Cell-Segmentatior assume that all input images are of the same shape!
"""

kernel_mode = False
debug = False

import sys
if kernel_mode:
    sys.path.insert(0, "../input/hpa-bestfitting-solution/src")
    sys.path.insert(0, "../input/hpa-cell-segmentation")

In [3]:
# !ls ../input/hpa-bestfitting-solution/

In [4]:
# !cp ../input/hpa-bestfitting-solution/densenet121-a639ec97.pth .
# !ls -la

In [5]:
# !pip install -q "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
# !pip install -q "../input/hpapytorchzoozip/pytorch_zoo-master"

In [6]:
import sys
import argparse
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import time
import random
import math
import pickle
from pickle import dump, load
import glob
import time
import collections

import torch
import torch.optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler
from torch.nn import DataParallel
import torch.nn.functional as F
from torch.autograd import Variable

from config.config import *
from utils.common_util import *
from networks.imageclsnet import init_network
from datasets.protein_dataset import ProteinDataset
from utils.augment_util import *
from datasets.tool import *

import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional import classification

from pycocotools import _mask as coco_mask
import typing as t
import base64
import zlib

import cv2
from PIL import Image
import imagehash

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

import gc
gc.enable()

rand_seed = 1120

print(f"PyTorch Version: {torch.__version__}")
print(f"PyTorch Lightning Version: {pl.__version__}")

run on 2fb1688be1a5
PyTorch Version: 1.6.0+cu101
PyTorch Lightning Version: 1.1.1


In [7]:
if kernel_mode:
    dataset_folder = "/kaggle/input/hpa-single-cell-image-classification"
    bestfitting_folder = "/kaggle/input/hpa-bestfitting-solution"
    test_image_folder = f"{dataset_folder}/test/"
    NUC_MODEL = "/kaggle/input/hpa-cell-segmentation/dpn_unet_nuclei_v1.pth"
    CELL_MODEL = "/kaggle/input/hpa-cell-segmentation/dpn_unet_cell_3ch_v1.pth"
else:
    dataset_folder = "/workspace/Kaggle/HPA/hpa_2020"
    bestfitting_folder = "/workspace/Github/HPA-competition-solutions/bestfitting"
    test_image_folder = f"{dataset_folder}/test/"
    NUC_MODEL = "/workspace/Github/HPA-Cell-Segmentation/dpn_unet_nuclei_v1.pth"
    CELL_MODEL = "/workspace/Github/HPA-Cell-Segmentation/dpn_unet_cell_3ch_v1.pth"

# image_size = 2048
# image_size = 768
crop_size = 512

batch_size = 8 if kernel_mode else 6
# batch_size = 4
num_workers = 2 if kernel_mode else 3

# scale_factor = 1.0
# scale_factor = 0.1
scale_factor = 0.25
confidence_threshold = 0.5

In [8]:
old_classes = {
    0: 'Nucleoplasm',
    1: 'Nuclear membrane',
    2: 'Nucleoli',
    3: 'Nucleoli fibrillar center',
    4: 'Nuclear speckles',
    5: 'Nuclear bodies',
    6: 'Endoplasmic reticulum',
    7: 'Golgi apparatus',
    8: 'Peroxisomes',
    9: 'Endosomes',
    10: 'Lysosomes',
    11: 'Intermediate filaments',
    12: 'Actin filaments',
    13: 'Focal adhesion sites',
    14: 'Microtubules',
    15: 'Microtubule ends',
    16: 'Cytokinetic bridge',
    17: 'Mitotic spindle',
    18: 'Microtubule organizing center',
    19: 'Centrosome',
    20: 'Lipid droplets',
    21: 'Plasma membrane',
    22: 'Cell junctions',
    23: 'Mitochondria',
    24: 'Aggresome',
    25: 'Cytosol',
    26: 'Cytoplasmic bodies',
    27: 'Rods & rings'
}
old_class_indices = {v: k for k, v in old_classes.items()}

# All label names in the public HPA and their corresponding index.
all_locations = dict({
    "Nucleoplasm": 0,
    "Nuclear membrane": 1,
    "Nucleoli": 2,
    "Nucleoli fibrillar center": 3,
    "Nuclear speckles": 4,
    "Nuclear bodies": 5,
    "Endoplasmic reticulum": 6,
    "Golgi apparatus": 7,
    "Intermediate filaments": 8,
    "Actin filaments": 9,
    "Focal adhesion sites": 9,
    "Microtubules": 10,
    "Mitotic spindle": 11,
    "Centrosome": 12,
    "Centriolar satellite": 12,
    "Plasma membrane": 13,
    "Cell Junctions": 13,
    "Mitochondria": 14,
    "Aggresome": 15,
    "Cytosol": 16,
    "Vesicles": 17,
    "Peroxisomes": 17,
    "Endosomes": 17,
    "Lysosomes": 17,
    "Lipid droplets": 17,
    "Cytoplasmic bodies": 17,
    "Rods & rings": 18,
    # markpeng
    "No staining": 18,
})

old_class_mappings = {}
for i, (k, v) in enumerate(old_class_indices.items()):
    if k in all_locations:
        old_class_mappings[v] = all_locations[k]
    else:
        # No staining
        old_class_mappings[v] = 18
assert len(old_class_mappings.values()) == len(old_classes.values())
print(old_class_mappings)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 17, 9: 17, 10: 17, 11: 8, 12: 9, 13: 9, 14: 10, 15: 18, 16: 18, 17: 11, 18: 18, 19: 12, 20: 17, 21: 13, 22: 18, 23: 14, 24: 15, 25: 16, 26: 17, 27: 18}


In [9]:
!ls {dataset_folder}

inference	       test	       train	  train_tfrecords
sample_submission.csv  test_tfrecords  train.csv


In [10]:
train_df = pd.read_csv(f"{dataset_folder}/train.csv")
submit_df = pd.read_csv(f"{dataset_folder}/sample_submission.csv")

In [11]:
print(train_df.shape)
train_df.head()

(21806, 2)


,ID,Label
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0


In [12]:
print(submit_df.shape, submit_df.ImageWidth.min(), submit_df.ImageWidth.max())
submit_df.head()

(559, 4) 1728 3072


,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==


In [13]:
colors = ["red", "green", "blue", "yellow"]

# Extract unique image IDs
# test_ids = [
#     f.split("/")[-1].replace("_red.png", "")
#     for f in glob.glob(f"{test_image_folder}/*_red.png")
# ]
# print(len(test_ids))
test_ids = submit_df["ID"].values.tolist()
print(len(test_ids))

# Estimated number of private test images (RGBY): 2236 x 2.3 ~= 5143 (for 9 hours we have 6.2 secs per image)
# Estimated number of private test images: 559 x 2.3 ~= 1286 (for 9 hours we have 25.2 secs per image)

559


## Utility Functions

In [14]:
# Reference: https://www.kaggle.com/dschettler8845/hpa-cellwise-classification-inference/notebook
def binary_mask_to_ascii(mask, mask_val=1):
    """Converts a binary mask into OID challenge encoding ascii text."""
    mask = np.where(mask == mask_val, 1, 0).astype(np.bool)

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
            f"encode_binary_mask expects a binary mask, received dtype == {mask.dtype}"
        )

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            f"encode_binary_mask expects a 2d mask, received shape == {mask.shape}"
        )

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode()


def rle_encoding(img, mask_val=1):
    """
    Turns our masks into RLE encoding to easily store them
    and feed them into models later on
    https://en.wikipedia.org/wiki/Run-length_encoding
    
    Args:
        img (np.array): Segmentation array
        mask_val (int): Which value to use to create the RLE
        
    Returns:
        RLE string
    
    """
    dots = np.where(img.T.flatten() == mask_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b

    return ' '.join([str(x) for x in run_lengths])


def rle_to_mask(rle_string, height, width):
    """ Convert RLE sttring into a binary mask 
    
    Args:
        rle_string (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array of the binary segmentation mask for a given cell
    """
    rows, cols = height, width
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    rle_pairs = np.array(rle_numbers).reshape(-1, 2)
    img = np.zeros(rows * cols, dtype=np.uint8)
    for index, length in rle_pairs:
        index -= 1
        img[index:index + length] = 255
    img = img.reshape(cols, rows)
    img = img.T
    return img


def create_segmentation_maps(list_of_image_lists, segmentator, batch_size=8):
    """ Function to generate segmentation maps using CellSegmentator tool 
    
    Args:
        list_of_image_lists (list of lists):
            - [[micro-tubules(red)], [endoplasmic-reticulum(yellow)], [nucleus(blue)]]
        batch_size (int): Batch size to use in generating the segmentation masks
        
    Returns:
        List of lists containing RLEs for all the cells in all images
    """

    all_mask_rles = {}
    for i in tqdm(range(0, len(list_of_image_lists[0]), batch_size),
                  total=len(list_of_image_lists[0]) // batch_size):

        # Get batch of images
        sub_images = [
            img_channel_list[i:i + batch_size]
            for img_channel_list in list_of_image_lists
        ]  # 0.000001 seconds

        # Do segmentation
        cell_segmentations = segmentator.pred_cells(sub_images)
        nuc_segmentations = segmentator.pred_nuclei(sub_images[2])

        # post-processing
        for j, path in enumerate(sub_images[0]):
            img_id = path.replace("_red.png", "").rsplit("/", 1)[1]
            nuc_mask, cell_mask = label_cell(nuc_segmentations[j],
                                             cell_segmentations[j])
            new_name = os.path.basename(path).replace('red', 'mask')
            all_mask_rles[img_id] = [
                rle_encoding(cell_mask, mask_val=k)
                for k in range(1,
                               np.max(cell_mask) + 1)
            ]
    return all_mask_rles


def get_img_list(img_dir, return_ids=False, sub_n=None):
    """ Get image list in the format expected by the CellSegmentator tool """
    if sub_n is None:
        sub_n = len(glob(img_dir + '/' + f'*_red.png'))
    if return_ids:
        images = [
            sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n]
            for c in ["red", "yellow", "blue"]
        ]
        return [
            x.replace("_red.png", "").rsplit("/", 1)[1] for x in images[0]
        ], images
    else:
        return [
            sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n]
            for c in ["red", "yellow", "blue"]
        ]


def get_contour_bbox_from_rle(
    rle,
    width,
    height,
    return_mask=True,
):
    """ Get bbox of contour as `xmin ymin xmax ymax`
    
    Args:
        rle (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array for a cell bounding box coordinates
    """
    mask = rle_to_mask(rle, height, width).copy()
    cnts = grab_contours(
        cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE))
    x, y, w, h = cv2.boundingRect(cnts[0])

    if return_mask:
        return (x, y, x + w, y + h), mask
    else:
        return (x, y, x + w, y + h)


def get_contour_bbox_from_raw(raw_mask):
    """ Get bbox of contour as `xmin ymin xmax ymax`
    
    Args:
        raw_mask (nparray): Numpy array containing segmentation mask information
    
    Returns:
        Numpy array for a cell bounding box coordinates
    """
    cnts = grab_contours(
        cv2.findContours(raw_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE))
    xywhs = [cv2.boundingRect(cnt) for cnt in cnts]
    xys = [(xywh[0], xywh[1], xywh[0] + xywh[2], xywh[1] + xywh[3])
           for xywh in xywhs]
    return sorted(xys, key=lambda x: (x[1], x[0]))


def pad_to_square(a):
    """ Pad an array `a` evenly until it is a square """
    if a.shape[1] > a.shape[0]:  # pad height
        n_to_add = a.shape[1] - a.shape[0]
        top_pad = n_to_add // 2
        bottom_pad = n_to_add - top_pad
        a = np.pad(a, [(top_pad, bottom_pad), (0, 0), (0, 0)], mode='constant')

    elif a.shape[0] > a.shape[1]:  # pad width
        n_to_add = a.shape[0] - a.shape[1]
        left_pad = n_to_add // 2
        right_pad = n_to_add - left_pad
        a = np.pad(a, [(0, 0), (left_pad, right_pad), (0, 0)], mode='constant')
    else:
        pass
    return a


def cut_out_cells(rgby,
                  rles,
                  resize_to=(256, 256),
                  square_off=True,
                  return_masks=False,
                  from_raw=True):
    """ Cut out the cells as padded square images 
    
    Args:
        rgby (np.array): 4 Channel image to be cut into tiles
        rles (list of RLE strings): List of run length encoding containing 
            segmentation mask information
        resize_to (tuple of ints, optional): The square dimension to resize the image to
        square_off (bool, optional): Whether to pad the image to a square or not
        
    Returns:
        list of square arrays representing squared off cell images
    """
    w, h = rgby.shape[:2]
    contour_bboxes = [
        get_contour_bbox(rle, w, h, return_mask=return_masks) for rle in rles
    ]
    if return_masks:
        masks = [x[-1] for x in contour_bboxes]
        contour_bboxes = [x[:-1] for x in contour_bboxes]

    arrs = [
        rgby[bbox[1]:bbox[3], bbox[0]:bbox[2], ...] for bbox in contour_bboxes
    ]
    if square_off:
        arrs = [pad_to_square(arr) for arr in arrs]

    if resize_to is not None:
        arrs = [
            cv2.resize(pad_to_square(arr).astype(np.float32),
                       resize_to,
                       interpolation=cv2.INTER_CUBIC) \
            for arr in arrs
        ]
    if return_masks:
        return arrs, masks
    else:
        return arrs


def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(
            ("Contours tuple must have length 2 or 3, "
             "otherwise OpenCV changed their cv2.findContours return "
             "signature yet again. Refer to OpenCV's documentation "
             "in that case"))

    # return the actual contours array
    return cnts

In [15]:
# https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/72534
def generate_hash(img_dir,
                  colors,
                  dataset='train',
                  imread_func=None,
                  is_update=False):
    meta = meta.copy()
    hash_maps = {}
    for color in colors:
        hash_maps[color] = []
        for idx in tqdm(range(len(meta)), desc='train %s' % color):
            img = imread_func(img_dir, meta.iloc[idx][ID], color)
            hash = imagehash.phash(img)
            hash_maps[color].append(hash)

    for color in colors:
        meta[color] = hash_maps[color]

    return meta


def calc_hash(params):
    color, threshold, base_test_hash1, base_test_hash2, test_ids1, test_ids2 = params

    test_hash1 = base_test_hash1.reshape(1, -1)  # 1*m

    test_idxes_list1 = []
    test_idxes_list2 = []
    hash_list = []

    step = 5
    for test_idx in tqdm(range(0, len(base_test_hash2), step), desc=color):
        test_hash2 = base_test_hash2[test_idx:test_idx + step].reshape(
            -1, 1)  # n*1
        hash = test_hash2 - test_hash1  # n*m
        test_idxes2, test_idxes1 = np.where(hash <= threshold)
        hash = hash[test_idxes2, test_idxes1]

        test_idxes2 = test_idxes2 + test_idx

        test_idxes_list1.extend(test_idxes1.tolist())
        test_idxes_list2.extend(test_idxes2.tolist())
        hash_list.extend(hash.tolist())

    df = pd.DataFrame({
        'Test1': test_ids1[test_idxes_list1],
        'Test2': test_ids2[test_idxes_list2],
        'Sim%s' % color[:1].upper(): hash_list
    })
    df = df[df['Test1'] != df['Test2']]
    return df

## Preprocessing

### Calculate Mean and Std From Training and Test Sets

In [16]:
# !python {bestfitting_folder}/src/data_process/s8_generate_images_mean_std.py \
#     --source /workspace/Kaggle/HPA/hpa_2020

In [17]:
total_mean = [0.081018, 0.052349, 0.054012, 0.08106] # rgby
total_std = [0.133235, 0.08948, 0.143813, 0.130265]

### Resize and Save All Images

In [18]:
script_path = "../input/hpa-bestfitting-solution/src/data_process" if kernel_mode else \
    "/workspace/Github/HPA-competition-solutions/bestfitting/src"

In [19]:
# !ls -la ../input/hpa-bestfitting-solution/src/data_process

In [20]:
# !cd {script_path} &&\
#     python s2_resize_png_image.py \
#     --source {dataset_folder} \
#     --dest /kaggle/working \
#     --dataset test \
#     --size {image_size}

## Dataset

In [21]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei


# Based on https://github.com/CellProfiling/HPA-competition-solutions/tree/master/bestfitting
class SegmentedProteinDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        img_dir=None,
        transform=None,
        return_label=True,
        in_channels=4,
        crop_size=0,
        random_crop=False,
    ):
        self.return_label = return_label
        self.in_channels = in_channels
        self.transform = transform
        self.crop_size = crop_size
        self.random_crop = random_crop

        self.segmentator = cellsegmentator.CellSegmentator(
            NUC_MODEL,
            CELL_MODEL,
            scale_factor=0.25,
            device="cuda",
            padding=True,
            multi_channel_model=True,
        )

        self.img_dir = img_dir
        self.img_ids = test_ids
        self.num = len(self.img_ids)

    def read_crop_img(self, img):
        random_crop_size = int(np.random.uniform(self.crop_size,
                                                 self.img_size))
        x = int(np.random.uniform(0, self.img_size - random_crop_size))
        y = int(np.random.uniform(0, self.img_size - random_crop_size))
        crop_img = img[x:x + random_crop_size, y:y + random_crop_size]
        return crop_img

    def read_rgby(self, img_dir, img_id, index):
        suffix = '.png'
        colors = ['red', 'green', 'blue', 'yellow']

        flags = cv2.IMREAD_GRAYSCALE
        rgby_img = [
            cv2.imread(opj(img_dir, img_id + '_' + color + suffix), flags)
            for color in colors
        ]
        #         for color in colors:
        #             print(opj(img_dir, img_id + '_' + color + suffix))
        rgby_img = np.stack(rgby_img, axis=-1)
        if self.random_crop and self.crop_size > 0:
            rgby_img = self.read_crop_img(rgby_img)

        return rgby_img

    def release_gpu(self):
        # Not work
        del self.segmentator
        torch.cuda.empty_cache()
        gc.collect()

    def __getitem__(self, index):
        img_id = self.img_ids[index]
        #         img_id = "0173029a-161d-40ef-af28-2342915b22fb"

        rgby_img = self.read_rgby(self.img_dir, img_id, index)
        if rgby_img[0] is None:
            print(self.img_dir, img_id)

        # print(rgby_img.shape)

        h, w = rgby_img.shape[:2]
        self.img_size = h

        if self.crop_size > 0:
            if self.crop_size != h or self.crop_size != w:
                resized_rgby_img = cv2.resize(rgby_img,
                                              (self.crop_size, self.crop_size),
                                              interpolation=cv2.INTER_LINEAR)
        else:
            if self.img_size != h or self.img_size != w:
                resized_rgby_img = cv2.resize(rgby_img,
                                              (self.img_size, self.img_size),
                                              interpolation=cv2.INTER_LINEAR)

        if self.transform is not None:
            resized_rgby_img = self.transform(resized_rgby_img)
        # rgby_img = rgby_img / 255.0

        # For nuclei
        nuc_segmentations = self.segmentator.pred_nuclei([rgby_img[:, :, 2]])

        # For full cells
        cell_segmentations = self.segmentator.pred_cells([[rgby_img[:, :, i]]
                                                          for i in [0, 3, 2]])

        # Extract cell masks
        _, full_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])

        #         print(full_mask.shape)
        full_mask = cv2.resize(full_mask, (self.crop_size, self.crop_size),
                               interpolation=cv2.INTER_NEAREST)

        del nuc_segmentations, cell_segmentations, rgby_img
        gc.collect()
        
        # print(np.min(full_mask), np.max(full_mask))
        cell_masks = [
            rle_encoding(full_mask, mask_val=k)
            for k in range(1,
                           np.max(full_mask) + 1)
        ]
        #         cell_masks = [
        #             rle_encoding(full_mask, mask_val=k)
        #             for k in range(1,
        #                            np.max(full_mask) + 1)
        #         ]
        if len(cell_masks) == 0:
            print(f"No cell masks found for {img_id}")

#         fig, ax = plt.subplots(figsize=(10, 10))
#         #         gbr_img = np.dstack([rgby_img[..., :i] for i in range(3)])
#         gbr_img = np.dstack([resized_rgby_img[..., :i] for i in [2, 1, 0]])
#         #         print(gbr_img.shape)
#         ax.imshow(gbr_img)
#         ax.imshow(full_mask, alpha=0.5)
#         plt.show()

#         print(resized_rgby_img.shape)

        resized_rgby_img = resized_rgby_img / 255.0
        resized_rgby_img = image_to_tensor(resized_rgby_img)

        if self.return_label:
            label = self.labels[index]
            return resized_rgby_img, cell_masks, label, img_id
        else:
            return resized_rgby_img, cell_masks, img_id


#         if self.return_label:
#             label = self.labels[index]
#             return rgby_img, cell_masks, label, img_id
#         else:
#             return rgby_img, cell_masks, img_id

    def __len__(self):
        return self.num

In [22]:
# tmp_dataset = SegmentedProteinDataset(
#     img_dir="/workspace/Kaggle/HPA/hpa_2020/inference/test/images_768",
#     img_size=768,
#     return_label=False,
#     in_channels=4,
#     transform=None,
#     crop_size=512,
#     random_crop=False,
# )
# tmpiter = iter(tmp_dataset)
# tmp_img, tmp_mask, tmp_index = next(tmpiter)
# print(tmp_img.shape, len(tmp_mask), tmp_index)

In [23]:
# fig, ax = plt.subplots(1, len(tmp_mask), figsize=(10, 15))
# for i in range(len(tmp_mask)):
#     tmp = np.copy(rle_to_mask(tmp_mask[i], 512, 512))
#     ax[i].imshow(tmp)
# plt.show()

## Load Pretrained Model from Bestfitting

In [24]:
datasets_names = ['test', 'val']
split_names = ['random_ext_folds5', 'random_ext_noleak_clean_folds5']
augment_list = ['default', 'flipud', 'fliplr','transpose', 'flipud_lr',
                'flipud_transpose', 'fliplr_transpose', 'flipud_lr_transpose']

parser = argparse.ArgumentParser(description='PyTorch Protein Classification')
parser.add_argument('--out_dir', type=str, help='destination where predicted result should be saved')
parser.add_argument('--gpu_id', default='0', type=str, help='gpu id used for predicting (default: 0)')
parser.add_argument('--arch', default='class_densenet121_dropout', type=str,
                    help='model architecture (default: class_densenet121_dropout)')
parser.add_argument('--num_classes', default=28, type=int, help='number of classes (default: 28)')
parser.add_argument('--in_channels', default=4, type=int, help='in channels (default: 4)')
parser.add_argument('--img_size', default=768, type=int, help='image size (default: 768)')
parser.add_argument('--crop_size', default=512, type=int, help='crop size (default: 512)')
parser.add_argument('--batch_size', default=32, type=int, help='train mini-batch size (default: 32)')
parser.add_argument('--workers', default=3, type=int, help='number of data loading workers (default: 3)')
parser.add_argument('--fold', default=0, type=int, help='index of fold (default: 0)')
parser.add_argument('--augment', default='default', type=str, help='test augmentation (default: default)')
parser.add_argument('--seed', default=100, type=int, help='random seed (default: 100)')
parser.add_argument('--seeds', default=None, type=str, help='predict seed')
parser.add_argument('--dataset', default='test', type=str, choices=datasets_names,
                    help='dataset options: ' + ' | '.join(datasets_names) + ' (default: test)')
parser.add_argument('--split_name', default='random_ext_folds5', type=str, choices=split_names,
                    help='split name options: ' + ' | '.join(split_names) + ' (default: random_ext_folds5)')
parser.add_argument('--predict_epoch', default=None, type=int, help='number epoch to predict')

_StoreAction(option_strings=['--predict_epoch'], dest='predict_epoch', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='number epoch to predict', metavar=None)

In [25]:
args = parser.parse_args([
    "--out_dir", "external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds",
    "--gpu_id", "0",
    "--arch", "class_densenet121_dropout",
#     "--img_size", str(image_size),
    "--crop_size", str(crop_size),
    "--seeds", str(rand_seed),
    "--batch_size", str(batch_size),
    "--fold", "0",
    "--augment", "default,flipud,fliplr,transpose,flipud_lr,flipud_transpose,fliplr_transpose,flipud_lr_transpose",
    "--dataset", "test"
])
args

Namespace(arch='class_densenet121_dropout', augment='default,flipud,fliplr,transpose,flipud_lr,flipud_transpose,fliplr_transpose,flipud_lr_transpose', batch_size=6, crop_size=512, dataset='test', fold=0, gpu_id='0', img_size=768, in_channels=4, num_classes=28, out_dir='external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds', predict_epoch=None, seed=100, seeds='1120', split_name='random_ext_folds5', workers=3)

In [26]:
# args.predict_epoch = 'final' if args.predict_epoch is None else '%03d' % args.predict_epoch
# network_path = opj(RESULT_DIR, 'models', args.out_dir, 'fold%d' % args.fold,
#                    '%s.pth' % args.predict_epoch)
network_path = f"{bestfitting_folder}/external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/fold0/final.pth"

# setting up the visible GPU
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

model_params = {}
model_params['architecture'] = args.arch
model_params['num_classes'] = args.num_classes
model_params['in_channels'] = args.in_channels
model_params['pretrained_path'] = f"{bestfitting_folder}"
model = init_network(model_params)

# log.write(">> Loading network:\n>>>> '{}'\n".format(network_path))
checkpoint = torch.load(network_path)
model.load_state_dict(checkpoint['state_dict'])
# log.write(">>>> loaded network:\n>>>> epoch {}\n".format(checkpoint['epoch']))

# moving network to gpu and eval mode
# model = DataParallel(model)
model.cuda()
model.eval()

>> Using pre-trained model.


DensenetClass(
  (backbone): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momen

## Inference

In [33]:
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=True,
    multi_channel_model=True,
)


def read_crop_img(img):
    random_crop_size = int(np.random.uniform(self.crop_size, self.img_size))
    x = int(np.random.uniform(0, self.img_size - random_crop_size))
    y = int(np.random.uniform(0, self.img_size - random_crop_size))
    crop_img = img[x:x + random_crop_size, y:y + random_crop_size]
    return crop_img


def read_rgby(
    img_dir,
    img_id,
    random_crop=False,
):
    suffix = '.png'
    colors = ['red', 'green', 'blue', 'yellow']

    flags = cv2.IMREAD_GRAYSCALE
    rgby_img = [
        cv2.imread(opj(img_dir, img_id + '_' + color + suffix), flags)
        for color in colors
    ]
    #         for color in colors:
    #             print(opj(img_dir, img_id + '_' + color + suffix))
    rgby_img = np.stack(rgby_img, axis=-1)
    if random_crop and crop_size > 0:
        rgby_img = read_crop_img(rgby_img)

    return rgby_img

please compile abn


In [34]:
def process_image(img_id, img_dir):
    rgby_img = read_rgby(img_dir, img_id)
    if rgby_img[0] is None:
        print(self.img_dir, img_id)

    h, w = rgby_img.shape[:2]

    if crop_size > 0:
        if crop_size != h or crop_size != w:
            resized_rgby_img = cv2.resize(rgby_img, (crop_size, crop_size),
                                          interpolation=cv2.INTER_LINEAR)

    # For nuclei
    nuc_segmentations = segmentator.pred_nuclei([rgby_img[:, :, 2]])

    # For full cells
    cell_segmentations = segmentator.pred_cells([[rgby_img[:, :, i]]
                                                 for i in [0, 3, 2]])

    # Extract cell masks
    _, full_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])

    full_mask = cv2.resize(full_mask, (crop_size, crop_size),
                           interpolation=cv2.INTER_NEAREST)

    del nuc_segmentations, cell_segmentations, rgby_img
    gc.collect()

    cell_masks = [
        rle_encoding(full_mask, mask_val=k)
        for k in range(1,
                       np.max(full_mask) + 1)
    ]
    if len(cell_masks) == 0:
        print(f"No cell masks found for {img_id}")

    resized_rgby_img = resized_rgby_img / 255.0
    resized_rgby_img = image_to_tensor(resized_rgby_img)

    return resized_rgby_img, cell_masks

In [35]:
def collate_cells_fn(x):
    images, masks = [], []
    # For each full image, extract cell images
    image = x[0]

    for rle_string in x[1]:
        cell_mask = rle_to_mask(rle_string, crop_size, crop_size)
        # Important: set 255 to 1
        cell_mask[cell_mask > 0] = 1

        cell_image = torch.clone(image)
        for i in range(4):
            cell_image[i, ...] = cell_image[i, ...] * cell_mask
        images.append(cell_image.unsqueeze(0))

        cell_mask = rle_encoding(cell_mask, mask_val=1)
        masks.append(cell_mask)

    images = torch.cat(images)
    return images, masks

In [36]:
all_probs = []
all_predictions = []
img_dir = "/kaggle/input/hpa-single-cell-image-classification/test" if kernel_mode \
        else "/workspace/Kaggle/HPA/hpa_2020/test"
for index, row in tqdm(submit_df.iterrows(), total=submit_df.shape[0]):
    id = row["ID"]
    width = row["ImageWidth"]
    height = row["ImageHeight"]

    cell_probs = []
    processed_count = 0
    images, masks = process_image(id, img_dir)
    images, masks = collate_cells_fn((images, masks))
    for batch_i in range(0, len(masks), batch_size):
        batch_images = images[batch_i:batch_i + batch_size, ...]
        batch_images = Variable(batch_images.cuda(), volatile=True)
        outputs = model(batch_images)
        logits = outputs

        probs = F.sigmoid(logits).data
        probs = probs.cpu().numpy().tolist()
        cell_probs += probs

        processed_count += len(probs)

    cell_probs = np.array(cell_probs).reshape(processed_count, -1)
    all_probs.append(cell_probs)
    
    masks = np.array(masks)

    if masks.shape[0] > 0:
        new_preds = np.zeros((cell_probs.shape[0], 19))
        for i in range(cell_probs.shape[0]):
            for j in range(28):
                new_class_i = old_class_mappings[j]
                # Take maximum prob.
                if cell_probs[i, j] > new_preds[i, new_class_i]:
                    new_preds[i, new_class_i] = cell_probs[i, j]

        # Generate RLE string for each cell mask
        # https://www.kaggle.com/dschettler8845/hpa-cellwise-classification-inference/notebook?scriptVersionId=55714434
        submit_strings = []
        for i in range(masks.shape[0]):
            confidence = new_preds[i, ...]
            mask = masks[i]

            mask = rle_to_mask(mask, crop_size, crop_size)
            # Important: set 255 to 1
            mask[mask > 0] = 1

            # Important: resize to orignal resolution to submit correct mask RLE string
            # https://www.kaggle.com/linshokaku/faster-hpa-cell-segmentation/comments#1251082
            #                 mask = cv2.resize(mask, (width, height),
            #                                   interpolation=cv2.INTER_LINEAR)
            mask = cv2.resize(mask, (width, height),
                              interpolation=cv2.INTER_NEAREST)
            #                 print(np.min(mask), np.max(mask))

            rle_string = binary_mask_to_ascii(mask, mask_val=1)
            # print(rle_string)

            #                 labels = sorted(
            #                     np.nonzero(preds > confidence_threshold)[0].tolist())
            # print(labels)
            #                 for l in labels:
            for l in range(19):
                submit_strings.append(f"{l} {confidence[l]:.6f} {rle_string}")

        if len(submit_strings) > 0:
            all_predictions.append(" ".join(submit_strings))
        else:
            all_predictions.append("")
    else:
        all_predictions.append("")

    if debug and index == batch_size - 1:
        break

    del images, masks, cell_probs
    gc.collect()

100%|██████████| 559/559 [52:59<00:00,  5.69s/it] 


In [37]:
all_probs = np.concatenate(all_probs, axis=0)
all_probs.shape

(10717, 28)

In [41]:
if debug:
    submit_df.iloc[:batch_size, :]["PredictionString"] = all_predictions
else:
    submit_df["PredictionString"] = all_predictions
submit_df

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 0.374412 eNq9VTtvwyAQ/ksQXyoPGTpkcGqEUWslpEI...
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,0 0.397984 eNoLCMhIMAgwDMg3NACDFIMYfwgrwcABwrD...
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,0 0.281442 eNqlkjELAyEMhf9SpBkcHG5wcEhPe3S4wcG...
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,0 0.328271 eNrtVbGOwyAM/SVbQiJDhg4dPCBa6XItAwM...
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 0.277026 eNq9U8sKAjEM/KVQJHrosaBEyoIs+GDJ1gr...
...,...,...,...,...
554,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 0.318939 eNqNUM0KwjAMfqWva2ehHjxNWEntQRgTGQ5...
555,feb955db-6c07-4717-a98b-92236c8e01d8,2048,2048,0 0.372260 eNqllMFu2zAMhl9JsjfDKTKghybZwVa4jVt...
556,fefb9bb7-934a-40d1-8d2f-210265857388,2048,2048,0 0.470812 eNrLMciwjEgzNEAGPiYQ2sMCQjskGBAAJj6...
557,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 0.453472 eNrlVcEKwyAM/aU0Gz159CS1BZHRQXFFBlu...


In [42]:
submit_df.to_csv("submission.csv", index=False)

In [43]:
def prob_to_result(probs, img_ids, th=0.5):
    predicted_probs = probs.copy()
    probs[np.arange(len(probs)), np.argmax(probs, axis=1)] = 1

    pred_list = []
    pred_list_new = []
    for line in probs:
        # Map old classes to new ones
        predicted_old_classes = sorted(
            list(set([i for i in np.nonzero(line > th)[0]])))
        predicted_new_classes = sorted(
            list(set([old_class_mappings[i]
                      for i in np.nonzero(line > th)[0]])))
        # print(predicted_classes)
        s = '|'.join([str(i) for i in predicted_old_classes])
        s_new = '|'.join([str(i) for i in predicted_new_classes])
        pred_list.append(s)
        pred_list_new.append(s_new)
    result_df = pd.DataFrame({
        # "ID": img_ids,
        "Predicted": pred_list,
        "Predicted_New": pred_list_new
    })
    return result_df

In [44]:
result_df = prob_to_result(all_probs, None, th=confidence_threshold)
result_df.to_csv("result_comparison.csv", index=False)
result_df.head()

,Predicted,Predicted_New
0,25,16
1,25,16
2,0,0
3,25,16
4,25,16


In [49]:
result_df.head(100)

,Predicted,Predicted_New
0,25,16
1,25,16
2,0,0
3,25,16
4,25,16
5,7,7
6,0,0
7,25,16
8,25,16
9,25,16


In [50]:
result_df["Predicted_New"].value_counts()

0           4361
16          3248
14           496
2            463
0|16         280
13           279
7            212
5            203
18           117
10           110
12           100
8             93
0|5           87
1             86
0|2           70
2|16          51
3             45
0|7           44
13|16         30
9             29
17            22
2|14          22
4             22
11|18         18
11            17
6             15
1|16          12
0|14          12
15            11
4|16          11
0|3           10
0|18           9
0|1            8
11|16          8
0|17           7
5|16           7
9|16           6
0|13           6
0|8            6
11|16|18       6
2|7            5
16|18          5
2|13           5
7|16           4
0|12           4
0|4            4
1|14           4
0|15           2
10|11          2
2|18           2
12|16          2
0|10           2
16|17          2
8|16           2
8|14           2
14|16          2
1|2            1
0|1|16         1
2|8|16        

In [45]:
# test_dataset.release_gpu()
# del model, test_dataset, test_loader
torch.cuda.empty_cache()
gc.collect()

49

In [46]:
# !rm densenet121-a639ec97.pth
# !rm -rf inference
# !ls -la

### EOF